In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

username = 'ubuntu'

params = { 'host': '3.16.56.135',
           'port': 5432
}

connection_string = f'postgres://{username}:{params["host"]}@{params["host"]}:{params["port"]}/faa'

In [3]:
engine = create_engine(connection_string)

In [4]:
db = pd.read_sql_query('select * from ord_all', engine)

In [5]:
db.shape

(1090649, 116)

In [6]:
cols = db.columns.tolist()

In [8]:
cols[-10:]

['Div5LongestGTime',
 'Div5WheelsOff',
 'Div5TailNum',
 'Unnamed: 109',
 'month_num',
 'yyyy',
 'cons_sent_um',
 'week_begin',
 'date',
 'wjfuelusgulf']

In [55]:
subset = db[['DepDel15', 'Distance', 'DepTime', 'DayOfWeek', 'Quarter', 'wjfuelusgulf', 'cons_sent_um']]

In [56]:
subset.shape

(1090649, 7)

In [52]:
db.[db.DepDel15 > .5]

SyntaxError: invalid syntax (<ipython-input-52-79da902bad36>, line 1)

In [57]:
mini_matx = pd.concat([pd.get_dummies(subset.DepDel15), subset], axis=1)

In [58]:
mini_matx = mini_matx.rename(columns={1.0: 'y'})

In [59]:
mini_matx = mini_matx.drop(columns=[0.0, 'DepDel15'])

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [87]:
mini_matx.shape

(1090649, 7)

In [88]:
mini_matx = mini_matx.dropna()

In [62]:
mini_matx.groupby('y').count()

,Distance,DepTime,DayOfWeek,Quarter,wjfuelusgulf,cons_sent_um
y,,,,,,
0,890491,871562,890491,890491,889559,890491
1,200158,200158,200158,200158,199969,200158


In [65]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

In [93]:
y = mini_matx['y']
X = mini_matx.iloc[:,1:]

In [95]:
y.shape

(1070601,)

In [98]:
X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, random_state=51)

over_smpl = RandomOverSampler(random_state=52)
X_O_smpld, y_O_smpld = over_smpl.fit_resample(X_tr, y_tr)

In [101]:
sclr = StandardScaler()
sclr.fit(X_O_smpld)

X_tr_scld = sclr.transform(X_O_smpld)
X_tst_scld = sclr.transform(X_tst)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [104]:
logr = LogisticRegression(C=1)

logr.fit(X_tr_scld, y_O_smpld)

logr.score(X_tst_scld, y_tst)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6083369761368349

In [115]:

#roc_auc_score(y_tst, logr.predict(y_tst))
type(X_tst_scld)

numpy.ndarray

In [116]:
X_tst_scld.shape

(267651, 6)

In [117]:
y_tst.shape

(267651,)

In [118]:
type(y_tst)

pandas.core.series.Series

In [119]:
type(y_O_smpld)

numpy.ndarray

In [121]:
y_O_smpld.shape

(1305702,)

In [127]:
logr.predict()

ValueError: Expected 2D array, got 1D array instead:
array=[0 0 0 ... 1 0 0].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [128]:
roc_auc_score(y_tst, logr.predict(X_tst_scld))

0.6146466223672911